In [2]:
# packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

Using the information on this website https://www.boxofficemojo.com/year/2002/?area=JP&grossesOption=calendarGrosses&sort=rank&sortDir=asc, 
I gathered the movies with top box office revenue in Japan for the last 20 years.

In [423]:
# import requests
# from bs4 import BeautifulSoup

# # Box Office Mojoのウェブサイトからデータをスクレイピングする関数
# def scrape_box_office_data():
#     base_url = "https://www.boxofficemojo.com/year/"
#     ranking = []

#     for year in range(2002, 2023):
#         url = f"{base_url}{year}/?area=JP&grossesOption=calendarGrosses&sort=rank&sortDir=asc"
#         response = requests.get(url)
#         soup = BeautifulSoup(response.content, "html.parser")
#         movie_rows = soup.select("#table tr")[1:]  # テーブルの行を取得 (ヘッダーを除く)

#         for row in movie_rows:
#             columns = row.select("td")
#             rank = columns[0].get_text(strip=True)
#             original_original_title = columns[1].get_text(strip=True)
#             total_gross = columns[7].get_text(strip=True)
#             gross = columns[5].get_text(strip=True)
#             release_date = columns[8].get_text(strip=True)
#             distributor = columns[9].get_text(strip=True)
            

#             ranking.append({
#                 "year": year,
#                 "rank": rank,
#                 "original_title": original_title,
#                 "gross": gross,
#                 "total_gross": total_gross,
#                 "release_date": release_date,
#                 "distributor": distributor
#             })

#     return ranking


# # メインの実行部分
# movies = scrape_box_office_data()
# movies

# # # 映画情報を表示
# # for movie in ranking:
# #     print("Year:", movie["year"])
# #     print("Rank:", movie["rank"])
# #     print("original_title:", movie["original_title"])
# #     print("Gross:", movie["gross"])
# #     print("Total Gross:", movie["total_gross"])
# #     print("Distributor:", movie["distributor"])
# #     print("------------------------")

# import pandas as pd
# ranking = pd.DataFrame(ranking)
# ranking.to_csv("ranking.csv", index=False)

# Ranking dataset
Top 118 - 200 movies of the year between 2002 and 2022 in Japan.

In [424]:
ranking = pd.read_csv("movies.csv")
ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3463 entries, 0 to 3462
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   year          3463 non-null   int64 
 1   rank          3463 non-null   int64 
 2   title         3463 non-null   object
 3   gross         3463 non-null   object
 4   total_gross   3463 non-null   object
 5   release_date  3463 non-null   object
 6   distributor   3463 non-null   object
dtypes: int64(2), object(5)
memory usage: 189.5+ KB


In [425]:
ranking.head()

,year,rank,title,gross,total_gross,release_date,distributor
0,2002,1,Star Wars: Episode II - Attack of the Clones,"$78,182,528","$78,182,528",Jul 13,-
1,2002,2,"Monsters, Inc.","$74,437,612","$74,437,612",Mar 2,-
2,2002,3,The Lord of the Rings: The Fellowship of the Ring,"$69,140,370","$69,140,370",Mar 2,-
3,2002,4,Harry Potter and the Chamber of Secrets,"$67,578,962","$142,786,414",Nov 23,-
4,2002,5,Spider-Man,"$58,925,689","$58,925,689",May 11,-


## Data Cleaning


In [426]:
# convert gross, total_gross to numeric
ranking["gross"] = ranking["gross"].str.replace("$", "").str.replace(",", "").astype(int)
ranking["total_gross"] = ranking["total_gross"].str.replace("$", "").str.replace(",", "").astype(int)

C:\Users\MICROS~1\AppData\Local\Temp/ipykernel_8828/2753252445.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ranking["gross"] = ranking["gross"].str.replace("$", "").str.replace(",", "").astype(int)
C:\Users\MICROS~1\AppData\Local\Temp/ipykernel_8828/2753252445.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ranking["total_gross"] = ranking["total_gross"].str.replace("$", "").str.replace(",", "").astype(int)


In [428]:
ranking.columns

Index(['year', 'rank', 'title', 'gross', 'total_gross', 'release_date',
       'distributor'],
      dtype='object')

In [430]:
# add a column "order". This column shows the order of the release date of the same movie.(For example, a movie is released in December, it is likely to show in the ranking of that year and the next year)
# orderという列を追加。titleとtotal_grossが重複している行にyearを昇順に割り当てる
ranking['order'] = ranking.sort_values(['title', 'total_gross'], ascending=[True, True]).groupby(['title', 'total_gross']).cumcount() + 1


In [431]:
# some movies' release_date is '-', meaning it is unknown, so I investigated online and manually added the release_date for those movies
ranking['release_date'].iloc[87] = '2002-11-09'
ranking['release_date'].iloc[109] = '2002-04-27'
ranking['release_date'].iloc[217] = '2003-12-20'
ranking['release_date'].iloc[288] = '2003-12-20'
ranking['release_date'].iloc[1160] = '2009-06-27'
ranking['release_date'].iloc[1205] = '2009-07-04'
ranking['release_date'].iloc[3061] = '2020-06-01'
ranking['release_date'].iloc[3139] = '2020-11-19'


c:\Users\Microsoft\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [432]:
# make a new column "release_date2" to show the release date of the first release
ranking['revised_release_date'] = ranking.release_date

for row in ranking.itertuples():
    if row.order == 2:
        # aggregate the "year" value to the release date
        ranking.revised_release_date[row.Index] = str(ranking.release_date[row.Index]) + " " + str(int(ranking.year[row.Index]) - 1)
    else:
        ranking.revised_release_date[row.Index] = str(ranking.release_date[row.Index]) + " " + str(int(ranking.year[row.Index]))

# Convert ['revised_release_date'] to datetime
ranking['revised_release_date'] = pd.to_datetime(ranking['revised_release_date']).astype('datetime64[ns]')

ranking['release_year'] = ranking['revised_release_date'].apply(lambda x: x.year)

C:\Users\MICROS~1\AppData\Local\Temp/ipykernel_8828/1239556119.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ranking.revised_release_date[row.Index] = str(ranking.release_date[row.Index]) + " " + str(int(ranking.year[row.Index]))
C:\Users\MICROS~1\AppData\Local\Temp/ipykernel_8828/1239556119.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ranking.revised_release_date[row.Index] = str(ranking.release_date[row.Index]) + " " + str(int(ranking.year[row.Index]) - 1)


In [434]:
# Movie original_title contains "Rereleased" refering to the movie being released again after several years from the original release
# rename the 'original_title' to 'original_title'
ranking.rename(columns={'title':'original_title'}, inplace=True)
ranking['revised_title'] = ranking['original_title'].str.lower().str.replace(" |3d|redux|re-release", "")

# re-release movies are not considered as original movies
ranking['re_released'] = ranking['original_title'].str.contains("Re-release", case=False)

for row in ranking.itertuples():
    if row.re_released == True:
        ranking.revised_title[row.Index] = ranking.revised_title[row.Index][:-4]
    

C:\Users\MICROS~1\AppData\Local\Temp/ipykernel_8828/2143727546.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  ranking['revised_title'] = ranking['original_title'].str.lower().str.replace(" |3d|redux|re-release", "")


C:\Users\MICROS~1\AppData\Local\Temp/ipykernel_8828/2143727546.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ranking.revised_title[row.Index] = ranking.revised_title[row.Index][:-4]


In [436]:
# Assign the id for each movie 

for index, row in ranking.iterrows():
    if row['order'] == 1:
        ranking.at[index, 'id'] = index
    else: # when duplicated
        original_title = row['original_title']
        year = row['release_year']
        matching_movie = ranking[(ranking['original_title'] == original_title) & (ranking['release_year'] == year) & (ranking['order'] == 1)]
        if not matching_movie.empty:
            order_1_id = matching_movie.iloc[0]['id']
            ranking.at[index, 'id'] = order_1_id
        else:
            ranking.at[index, 'id'] = None  # もしくは適切な値を設定する


In [437]:
# Ranking dataset is ready
ranking.to_csv('revised_ranking.csv', index=False)

# Movie list dataset

ranking (revised_ranking.csv) is a list of Top movies from 2002 to 2022 in Japan. Since some movies were released in the end of year (November, December...), they appear twice in the ranking, in the year they were released and in the next year.

Now, I want to make a dataset of unique movies and merge with imdb dataset to get more useful information about the movies such as genre, rating, budget etc...

In [438]:
movies = ranking.drop_duplicates(subset=['id'])
movies.drop(['year', 'rank', 'order', 'release_date'], axis=1, inplace=True)
# Replace column names
movies.rename(columns={'revised_release_date':'JP_release_date'}, inplace=True)
movies.rename(columns={'release_year':'JP_release_year'}, inplace=True)

c:\Users\Microsoft\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
c:\Users\Microsoft\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


# update.csv (imdb dataset from Kaggle)
I will merge this detailed dataset with the movies dataset I prepared based on the top movies in Japan.
https://www.kaggle.com/datasets/akshaypawar7/millions-of-movies

In [439]:
update = pd.read_csv("update.csv")
update

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
0,758323,The Pope's Exorcist,Horror-Mystery-Thriller,en,Father Gabriele Amorth Chief Exorcist of the V...,5953.227,Screen Gems-2.0 Entertainment-Jesus & Mary-Wor...,2023-04-05,18000000.0,6.567582e+07,103.0,Released,Inspired by the actual files of Father Gabriel...,7.433,545.0,Russell Crowe-Daniel Zovatto-Alex Essoe-Franco...,spain-rome italy-vatican-pope-pig-possession-c...,/9JBEPLTPSm0d1mbEcLxULjJq9Eh.jpg,/hiHGRbyTcbZoLsYYkO4QiCLYe34.jpg,713704-296271-502356-1076605-1084225-1008005-9...
1,667538,Transformers: Rise of the Beasts,Action-Adventure-Science Fiction,en,When a new threat capable of destroying the en...,5409.104,Skydance-Paramount-di Bonaventura Pictures-Bay...,2023-06-06,200000000.0,4.070455e+08,127.0,Released,Unite or fall.,7.340,1007.0,Anthony Ramos-Dominique Fishback-Luna Lauren V...,peru-alien-end of the world-based on cartoon-b...,/gPbM0MK8CP8A174rmUwGsADNYKD.jpg,/woJbg7ZqidhpvqFGGMRhWQNoxwa.jpg,496450-569094-298618-385687-877100-598331-4628...
2,640146,Ant-Man and the Wasp: Quantumania,Action-Adventure-Science Fiction,en,Super-Hero partners Scott Lang and Hope van Dy...,4425.387,Marvel Studios-Kevin Feige Productions,2023-02-15,200000000.0,4.757662e+08,125.0,Released,Witness the beginning of a new dynasty.,6.507,2811.0,Paul Rudd-Evangeline Lilly-Jonathan Majors-Kat...,hero-ant-sequel-superhero-based on comic-famil...,/qnqGbB22YJ7dSs4o6M7exTpNxPz.jpg,/m8JTwHFwX7I7JY5fPe4SjqejWag.jpg,823999-676841-868759-734048-267805-965839-1033...
3,677179,Creed III,Drama-Action,en,After dominating the boxing world Adonis Creed...,3994.342,Metro-Goldwyn-Mayer-Proximity Media-Balboa Pro...,2023-03-01,75000000.0,2.690000e+08,116.0,Released,You can't run from your past.,7.262,1129.0,Michael B. Jordan-Tessa Thompson-Jonathan Majo...,philadelphia pennsylvania-husband wife relatio...,/cvsXj3I9Q2iyyIo95AecSd1tad7.jpg,/5i6SjyDbDWqyun8klUuCxrlFbyw.jpg,965839-267805-943822-842942-1035806-823999-107...
4,502356,The Super Mario Bros. Movie,Animation-Family-Adventure-Fantasy-Comedy,en,While working underground to fix a water main ...,3859.926,Universal Pictures-Illumination-Nintendo,2023-04-05,100000000.0,1.278767e+09,92.0,Released,NaN,7.764,4042.0,Chris Pratt-Charlie Day-Anya Taylor-Joy-Jack B...,video game-gorilla-plumber-magic mushroom-anth...,/qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg,/2klQ1z1fcHGgQPevbEQdkCnzyuS.jpg,713704-385687-640146-60898-758323-1008005-4935...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722766,869220,Quicksand,NaN,en,An elderly gentleman attempts to retrieve forg...,0.600,NaN,NaN,0.0,0.000000e+00,8.0,Released,NaN,0.000,0.0,Roy Oppenheim-Walter Oppenheim,NaN,NaN,NaN,NaN
722767,710957,"Morning Musume. 2005 Summer ""Baribari Kyoushit...",NaN,ja,7th Gen Debuts in this Morning Musume. concert...,0.600,NaN,2005-11-27,0.0,0.000000e+00,107.0,Released,NaN,0.000,0.0,Yoshizawa Hitomi-Takahashi Ai-Konno Asami-Ogaw...,NaN,/mUpY1xq7mm3IZE3kxhEhP5nmDk0.jpg,/qqh84hkis13VF9WRSc2Njvj3NyM.jpg,NaN
722768,857920,The Ultimate Story Of the Spitfire,NaN,en,The Supermarine Spitfire's deadly duels over S...,0.600,NaN,2004-01-01,0.0,0.000000e+00,52.0,Released,NaN,0.000,0.0,NaN,NaN,/baRetWGXcRAEEpeMv8Wz1mYN6CM.jpg,NaN,NaN
722769,892477,The Drag Phenomenon,NaN,es,With more than 20 years of celebration the Dra...,0.600,NaN,2021-11-06,0.0,0.000000e+00,26.0,Released,NaN,0.000,0.0,NaN,NaN,/3p6o6bUwLIVhotMMEVe89CTISZg.jpg,/pCxXXZNkYAbCpElBHQaQz1FSl7r.jpg,NaN


In [440]:
# delete duplicates
update = update.drop_duplicates(subset=['title', 'release_date'], keep='first')

# change original_title to better merge two datasets
update['revised_title'] = update['title'].str.lower().str.replace(' ', '')

# drop na
# update.dropna(subset=['original_title'], inplace=True)

# drop movies whose status is not released
update = update[update['status'] == 'Released']

# change release_date to datetime
update['release_date'] = pd.to_datetime(update['release_date'])

C:\Users\MICROS~1\AppData\Local\Temp/ipykernel_8828/1753924428.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  update['revised_title'] = update['title'].str.lower().str.replace(' ', '')


# Merge movies & update



In [441]:
movies.columns

Index(['original_title', 'gross', 'total_gross', 'distributor',
       'JP_release_date', 'JP_release_year', 'revised_title', 're_released',
       'id'],
      dtype='object')

In [564]:
# merge two datasets based on the movies' original_title
merged = pd.merge(movies, update, how='left', left_on='revised_title', right_on='revised_title', suffixes=('_movies', '_tmdb'), indicator=False)
merged.drop(['poster_path', 'backdrop_path', 'recommendations'], axis=1, inplace=True)

In [565]:
# When there are more then one movies with the same name, we need to use the year to distinguish them.
merged['date_difference'] = abs(merged['JP_release_date'] - merged['release_date'])
merged = merged.sort_values(['original_title', 'date_difference'], ascending=[False, True])


In [566]:
# Drop duplicates 
# (1) In case of "re_released" == 1, I manually checked the data and detected the correct one
# Akira: drop the rows whose release_date are not 1988
merged.drop(merged[(merged['original_title']=="Akira") & (merged['release_date']!="1988-07-16")].index, inplace=True)

# Avatar: drop the rows whose revenue are 0
merged.drop(merged[(merged['original_title']=="Avatar") & (merged['revenue']==0)].index, inplace=True)

# Dream Catcher: drop the rows with revenue 0
merged.drop(merged[(merged['original_title']=="Dreamcatcher") & (merged['revenue']==0)].index, inplace=True)

# Gold: drop the rows with revenue 0 and release_date is not 2016
merged.drop(merged[(merged['original_title']=="Gold") & (merged['revenue']==0)].index, inplace=True)
merged.drop(merged[(merged['original_title']=="Gold") & (merged['release_date']!="2016-02-03")].index, inplace=True)

# greenzone: drop the rows with revenue 0
merged.drop(merged[(merged['revised_title']=="greenzone") & (merged['revenue']==0)].index, inplace=True)

# josee,thetigerandthefish: drop the row with release_date not 2020
merged.drop(merged[(merged['revised_title']=="josee,thetigerandthefish") & (merged['release_date']!="2020-12-25")].index, inplace=True)

# kiki'sdeliveryservice: drop the row with revenue 0
merged.drop(merged[(merged['revised_title']=="kiki'sdeliveryservice") & (merged['revenue']==0)].index, inplace=True)

# premonition: drop the row with revenue 0
merged.drop(merged[(merged['revised_title']=="premonition") & (merged['revenue']==0)].index, inplace=True)

# taxi: drop the row with release_date not 2004
merged.drop(merged[(merged['revised_title']=="taxi") & (merged['release_date']!="2004-10-06")].index, inplace=True)

# thehangover: drop the row with revenue 0
merged.drop(merged[(merged['revised_title']=="thehangover") & (merged['revenue']==0)].index, inplace=True)

# thelaststation: drop the row with release_date not 2009
merged.drop(merged[(merged['revised_title']=="thelaststation") & (merged['release_date']!="2009-09-04")].index, inplace=True)

# thenewworld: drop the row with release_date not 2005
merged.drop(merged[(merged['revised_title']=="thenewworld") & (merged['release_date']!="2005-12-25")].index, inplace=True)

# whisperoftheheart: drop the row with release_date not 2022
merged.drop(merged[(merged['revised_title']=="whisperoftheheart") & (merged['release_date']!="2022-10-14")].index, inplace=True)

# (2) In case of "re_released" == 0, keep the row with the least date_difference
merged = merged.sort_values(by=['id_movies', 'date_difference'], ascending=True)
merged.drop_duplicates(subset=['id_movies'], keep='first', inplace=True)

# Now all duplicates were dropped

## Cleaning the merged dataset
There are 691 movies that didn't match with any of the movies in the IMDB dataset. 
I will add some information, especially genre, country, and language, to these movies.

8/1やることの記録
* リリースデートの違いとrevenue, budgetがゼロかどうかなど確認して重複を消す
* keywordsを用いてジャンルの分類に使えるかも
* imdb idと一致していない気がするので確認
* 

In [445]:
merged.isna().sum()

original_title             0
gross                      0
total_gross                0
distributor                0
JP_release_date            0
JP_release_year            0
revised_title              0
re_released                0
id_movies                  1
id_tmdb                  691
title                    691
genres                   732
original_language        691
overview                 702
popularity               691
production_companies     884
release_date             692
budget                   691
revenue                  691
runtime                  694
status                   691
tagline                 1599
vote_average             691
vote_count               691
credits                  724
keywords                1054
date_difference          692
dtype: int64

In [567]:
# Gekijouban/eiga is used in Japanese movies' title so I will change the original language as "ja"
merged.loc[merged['original_title'].str.contains('gekijouban|eiga|the movie|monogatari', case=False), 'original_language'] = 'ja'

# watashi/boku is used in Japanese movies' title so I will change the original language as "ja"
merged.loc[merged['original_title'].str.contains('watashi|boku|atashi|anata|kimi', case=False), 'original_language'] = 'ja'

# movies by "Aniplex" are Japanese animation movies so I will change the original language as "ja" and the genres as "Animation"
merged.loc[merged['distributor'].str.contains('Aniplex', case=False), 'original_language'] = 'ja'
merged.loc[merged['distributor'].str.contains('Aniplex', case=False), 'genres'] = 'Animation'

# The movies whose original_title include "kureyon" or "shinchan" are "kureyon shinchan" franchise so I will change the genres as "Animation, Comedy, Family"
merged.loc[merged['original_title'].str.contains('shinchan|kureyon|shin-chan|shin chan', case=False), 'genres'] = 'Animation, Comedy, Family'

# Also change the original_language to "ja"
merged.loc[merged['original_title'].str.contains('shinchan|kureyon|shin-chan|shin chan', case=False), 'original_language'] = 'ja'

# The movies whose original_title include "doraemon" are "doraemon" franchise so I will change the genres as "Animation, Family"
merged.loc[merged['original_title'].str.contains('doraemon', case=False), 'genres'] = 'Animation, Family'

# Also change the original_language to "ja"
merged.loc[merged['original_title'].str.contains('doraemon', case=False), 'original_language'] = 'ja'
# The movies whose original_title include "kamen rider" are "kamen rider" franchise so I will change the genres as "Action, Adventure"
merged.loc[merged['original_title'].str.contains('kamen rider', case=False), 'genres'] = 'Action, Adventure'

# Also change the original_language to "ja"
merged.loc[merged['original_title'].str.contains('kamen rider', case=False), 'original_language'] = 'ja'

# The movies whose original_title include "pretty cure" are "pretty cure" franchise so I will change the genres as "Animation, Adventure, Family"
merged.loc[merged['original_title'].str.contains('pretty cure|purikyua', case=False), 'genres'] = 'Animation, Adventure, Family'

# Also change the original_language to "ja"
merged.loc[merged['original_title'].str.contains('pretty cure|purikyua', case=False), 'original_language'] = 'ja'
# "tsuribaka" movies are "tsuribaka" franchise so I will change the genres as "Drama"
merged.loc[merged['original_title'].str.contains('tsuribaka', case=False), 'genres'] = 'Drama' 
merged.loc[merged['original_title'].str.contains('tsuribaka', case=False), 'original_language'] = 'ja'
# "pokémon" movies are "pokémon" franchise so I will change the genres as "Animation, Action, Adventure", 
merged.loc[merged['original_title'].str.contains('pokémon', case=False), 'genres'] = 'Animation, Action, Adventure, Family'
merged.loc[merged['original_title'].str.contains('pokémon', case=False), 'original_language'] = 'ja'

# but "Detective Pikachu" is not "Animation" so I will change the genres as "Action, Adventure, Comedy"
merged.loc[merged['original_title'].str.contains('detective pikachu', case=False), 'genres'] = 'Action, Adventure, Family'
merged.loc[merged['original_title'].str.contains('detective pikachu', case=False), 'original_language'] = 'en'

# harry potter movies are "harry potter" franchise so I will change the genres as "Adventure, Fantasy" and the language as "en"
merged.loc[merged['original_title'].str.contains('harry potter', case=False), 'genres'] = 'Adventure, Fantasy'
merged.loc[merged['original_title'].str.contains('harry potter', case=False), 'original_language'] = 'en'
# spy kids movies are "spy kids" franchise so I will change the genres as "Action, Adventure, Comedy, family" and the language as "en"
merged.loc[merged['original_title'].str.contains('spy kids', case=False), 'genres'] = 'Action, Adventure, Comedy, Family'
merged.loc[merged['original_title'].str.contains('spy kids', case=False), 'original_language'] = 'en'
# one piece movies are "one piece" franchise so I will change the genres as "Animation, Action, Adventure" and the language as "ja"
merged.loc[merged['original_title'].str.contains('one piece', case=False), 'genres'] = 'Animation, Action, Adventure'
merged.loc[merged['original_title'].str.contains('one piece', case=False), 'original_language'] = 'ja'
# "naruto" movies are "naruto" franchise so I will change the genres as "Animation, Action, Adventure" and the language as "ja"
merged.loc[merged['original_title'].str.contains('naruto', case=False), 'genres'] = 'Animation, Action, Adventure'
merged.loc[merged['original_title'].str.contains('naruto', case=False), 'original_language'] = 'ja'
# "inuyasha" movies are "inuyasha" franchise so I will change the genres as "Animation, Action, Adventure" and the language as "ja"
merged.loc[merged['original_title'].str.contains('inuyasha', case=False), 'genres'] = 'Animation, Action, Adventure'
merged.loc[merged['original_title'].str.contains('inuyasha', case=False), 'original_language'] = 'ja'
# "yokai" or "yo-kai" are "yokai watch" franchise so I will change the genres as "Animation, Family, Adventure" and the language as "ja"

merged.loc[merged['original_title'].str.contains('yokai|yo-kai', case=False), 'genres'] = 'Animation, Family, Adventure'
merged.loc[merged['original_title'].str.contains('yokai|yo-kai', case=False), 'original_language'] = 'ja'

# "onmyoji" change the genres as "Horror" and the language as "ja"

merged.loc[merged['original_title'].str.contains('onmyoji', case=False), 'genres'] = 'Horror'
merged.loc[merged['original_title'].str.contains('onmyoji', case=False), 'original_language'] = 'ja'
# gintama change the genres as "Animation", Comedy"

merged.loc[merged['original_title'].str.contains('gintama', case=False), 'genres'] = 'Animation, Comedy'
merged.loc[merged['original_title'].str.contains('gintama', case=False), 'original_language'] = 'ja'
# "sentai" change the genre as "Action, Adventure, Family"
merged.loc[merged['original_title'].str.contains('sentai', case=False), 'genres'] = 'Action, Adventure, Family'
merged.loc[merged['original_title'].str.contains('sentai', case=False), 'original_language'] = 'ja'

# yu-gi-oh change the genre as "Animation, Action, Adventure"

merged.loc[merged['original_title'].str.contains('yu-gi-oh', case=False), 'genres'] = 'Animation, Action, Adventure'
merged.loc[merged['original_title'].str.contains('yu-gi-oh', case=False), 'original_language'] = 'ja'
# umizaru
merged.loc[merged['original_title'].str.contains('umizaru', case=False), 'genres'] = 'Action, Drama'
merged.loc[merged['original_title'].str.contains('umizaru', case=False), 'original_language'] = 'ja'
# Utapri
merged.loc[merged['original_title'].str.contains('utapri', case=False), 'genres'] = 'Animation, Music, Drama'
merged.loc[merged['original_title'].str.contains('utapri', case=False), 'original_language'] = 'ja'
# Ushijima
merged.loc[merged['original_title'].str.contains('ushijima', case=False), 'genres'] = 'Drama'
merged.loc[merged['original_title'].str.contains('ushijima', case=False), 'original_language'] = 'ja'
# kenshin
merged.loc[merged['original_title'].str.contains('rurouni', case=False), 'genres'] = 'Action, Adventure, Drama'
merged.loc[merged['original_title'].str.contains('rurouni', case=False), 'original_language'] = 'ja'
# Ultraman
merged.loc[merged['original_title'].str.contains('ultraman', case=False), 'genres'] = 'Action, Adventure'
merged.loc[merged['original_title'].str.contains('ultraman', case=False), 'original_language'] = 'ja'
# Anpanman
merged.loc[merged['original_title'].str.contains('anpanman', case=False), 'genres'] = 'Animation, Family'
merged.loc[merged['original_title'].str.contains('anpanman', case=False), 'original_language'] = 'ja'
# Star Wars
merged.loc[merged['original_title'].str.contains('star wars', case=False), 'genres'] = 'Adventure, Sci-Fi'
merged.loc[merged['original_title'].str.contains('star wars', case=False), 'original_language'] = 'en'
# Space Battleship Yamato
merged.loc[merged['original_title'].str.contains('space battleship yamato', case=False), 'genres'] = 'Animation, Science Fiction'
merged.loc[merged['original_title'].str.contains('space battleship yamato', case=False), 'original_language'] = 'ja'
# X-Men
merged.loc[merged['original_title'].str.contains('X-Men', case=False), 'genres'] = 'Action, Adventure, Science Fiction'
merged.loc[merged['original_title'].str.contains('X-Men', case=False), 'original_language'] = 'en'
# Bleach
merged.loc[merged['original_title'].str.contains('bleach', case=False), 'genres'] = 'Anime, Action, Adventure'
# merged.loc[merged['original_title'] == 'Bleach'] = 'Action, Fantasy, Adventure'
merged.loc[merged['original_title'].str.contains('bleach', case=False), 'original_language'] = 'ja'
# Detective Conan
merged.loc[merged['original_title'].str.contains('detective conan', case=False), 'genres'] = 'Animation, Mystery, Crime'
merged.loc[merged['original_title'].str.contains('detective conan', case=False), 'original_language'] = 'ja'

# Confidence man
merged.loc[merged['original_title'].str.contains('confidence man', case=False), 'genres'] = 'Comedy, Crime'
merged.loc[merged['original_title'].str.contains('confidence man', case=False), 'original_language'] = 'ja'

# attack on titan
merged.loc[merged['original_title'].str.contains('attack on titan', case=False), 'genres'] = 'Action, Adventure'
merged.loc[merged['original_title'].str.contains('attack on titan', case=False), 'original_language'] = 'ja'

In [616]:
# Manually insert the genre and language information for the movies with a big box office revenue
# Red Cliff II -> Action, Adventure, Drama, History, War
merged.loc[merged['original_title'] == 'Red Cliff II', 'genres'] = 'Action, Adventure, Drama, History, War'
merged.loc[merged['original_title'] == 'Red Cliff II', 'original_language'] = 'zh'
# Boys Over Flowers -> Romance, Drama
merged.loc[merged['original_title'].str.contains('boys over flowers', case=False), 'genres'] = 'Romance, Drama'
merged.loc[merged['original_title'].str.contains('boys over flowers', case=False), 'original_language'] = 'ja'
# # Bayside shakedown -> 'Action, Comedy, Drama, Crime'
merged.loc[merged['original_title'].str.contains('bayside shakedown', case=False), 'genres'] = 'Action, Comedy, Drama, Crime'
merged.loc[merged['original_title'].str.contains('bayside shakedown', case=False), 'original_language'] = 'ja'
# negotiator -> 'Action'
merged.loc[merged['original_title'].str.contains('Mashita Masayoshi', case=False), 'genres'] = 'Action'
merged.loc[merged['original_title'].str.contains('Mashita Masayoshi', case=False), 'original_language'] = 'ja'
# muroi shinji -> 'Comedy, Crime, Mystery'
merged.loc[merged['original_title'].str.contains('muroi shinji', case=False), 'genres'] = 'Comedy, Crime, Mystery'
merged.loc[merged['original_title'].str.contains('muroi shinji', case=False), 'original_language'] = 'ja'
# otoko-tachi no yamato -> 'Drama, War'
merged.loc[merged['original_title'].str.contains('otoko-tachi no yamato', case=False), 'genres'] = 'Drama, War'
merged.loc[merged['original_title'].str.contains('otoko-tachi no yamato', case=False), 'original_language'] = 'ja'
# Doomsday: The Sinking of Japan -> 'Action, Drama, Science Fiction' -> 'Adventure, Drama, Sci-Fi, Thriller'
merged.loc[merged['original_title'].str.contains('doomsday: the sinking of japan', case=False), 'genres'] = 'Adventure, Drama, Sci-Fi, Thriller'
merged.loc[merged['original_title'].str.contains('doomsday: the sinking of japan', case=False), 'original_language'] = 'ja'
# The Adventures of Super Monkey -> 'Action, Adventure, Fantasy'
merged.loc[merged['original_title'].str.contains('the adventures of super monkey', case=False), 'genres'] = 'Action, Adventure, Fantasy'
merged.loc[merged['original_title'].str.contains('the adventures of super monkey', case=False), 'original_language'] = 'ja'

# Escape Captivity -> 'Thriller
merged.loc[merged['original_title'].str.contains('escape captivity', case=False), 'genres'] = 'Thriller'
merged.loc[merged['original_title'].str.contains('escape captivity', case=False), 'original_language'] = 'ja'

# Rookies -> 'Drama'
merged.loc[merged['original_title'].str.contains('rookies the movie', case=False), 'genres'] = 'Drama'
merged.loc[merged['original_title'].str.contains('rookies the movie', case=False), 'original_language'] = 'ja'

# Amalfi -> 'Mystery, Thriller, Crime'
merged.loc[merged['original_title'].str.contains('amalfi: rewards of the goddess', case=False), 'genres'] = 'Mystery, Thriller, Crime'
merged.loc[merged['original_title'].str.contains('amalfi: rewards of the goddess', case=False), 'original_language'] = 'ja'

# SP the motion -> "Drama, Crime"
merged.loc[merged['original_title'].str.contains('sp: the motion picture', case=False), 'genres'] = 'Drama, Crime'
merged.loc[merged['original_title'].str.contains('sp: the motion picture', case=False), 'original_language'] = 'ja'

# Tenpesuto -> "History, Drama"
merged.loc[merged['original_title'].str.contains('tenpesuto', case=False), 'genres'] = 'History, Drama'
merged.loc[merged['original_title'].str.contains('tenpesuto', case=False), 'original_language'] = 'ja'

# From today, it's my turn!! -> "Comedy, Drama"
merged.loc[merged['original_title'].str.contains('from today, it\'s my turn!!', case=False), 'genres'] = 'Comedy, Drama'
merged.loc[merged['original_title'].str.contains('from today, it\'s my turn!!', case=False), 'original_language'] = 'ja'

# F9: The Fast Saga -> 'Action, Adventure'
merged.loc[merged['original_title'].str.contains('f9: the fast saga', case=False), 'genres'] = 'Action, Adventure'
merged.loc[merged['original_title'].str.contains('f9: the fast saga', case=False), 'original_language'] = 'en'

# Dogwood tree -> "Drama, Romance"
merged.loc[merged['original_title'].str.contains('dogwood tree', case=False), 'genres'] = 'Drama, Romance'
merged.loc[merged['original_title'].str.contains('dogwood tree', case=False), 'original_language'] = 'ja'

# Hit me Anyone -> "Comedy, "Drama"
merged.loc[merged['original_title'].str.contains('hit me anyone one more time', case=False), 'genres'] = 'Comedy, Drama'
merged.loc[merged['original_title'].str.contains('hit me anyone one more time', case=False), 'original_language'] = 'ja'

# Dragon Ball Z -> "Action, Adventure, Animation"
merged.loc[merged['original_title'].str.contains('dragon ball', case=False), 'genres'] = 'Action, Adventure, Animation'
merged.loc[merged['original_title'].str.contains('dragon ball', case=False), 'original_language'] = 'ja'

# Assassination classroom -> "Action, Adventure, Comedy, Drama, Science Fiction"
merged.loc[merged['original_title'].str.contains('assassination classroom', case=False), 'genres'] = 'Action, Adventure, Comedy, Drama, Science Fiction'
merged.loc[merged['original_title'].str.contains('assassination classroom', case=False), 'original_language'] = 'ja'

# Death Note: L change the World -> "Crime, Dra,a, Thriller"
merged.loc[merged['original_title'].str.contains('death note: l change the world', case=False), 'genres'] = 'Crime, Drama, Thriller'
merged.loc[merged['original_title'].str.contains('death note: l change the world', case=False), 'original_language'] = 'ja'

# New interpretation: -> "Comedy, History"
merged.loc[merged['original_title'].str.contains('new interpretation', case=False), 'genres'] = 'Comedy, History'
merged.loc[merged['original_title'].str.contains('new interpretation', case=False), 'original_language'] = 'ja'

# Mt.Tsurugidake -> "Drama, Adventure"
merged.loc[merged['original_title'].str.contains('mt. tsurugidake', case=False), 'genres'] = 'Drama, Adventure'
merged.loc[merged['original_title'].str.contains('mt. tsurugidake', case=False), 'original_language'] = 'ja'

# Nada so so -> "Drama, Romance"
merged.loc[merged['original_title'].str.contains('nada so so', case=False), 'genres'] = 'Drama, Romance'
merged.loc[merged['original_title'].str.contains('nada so so', case=False), 'original_language'] = 'ja'

# The Blind Swordsman -> "Adventure, Drama, Action"
merged.loc[merged['original_title'].str.contains('the blind swordsman', case=False), 'genres'] = 'Adventure, Drama, Action'
merged.loc[merged['original_title'].str.contains('the blind swordsman', case=False), 'original_language'] = 'ja'

In [623]:
merged.loc[(merged['original_title'].str.contains('ô', case=False)) & (merged['original_language'].isna()), 'original_language'] = 'ja'

In [625]:
merged[merged['original_language'].isna()].sort_values(by='gross', ascending=False)

,original_title,gross,total_gross,distributor,JP_release_date,JP_release_year,revised_title,re_released,id_movies,id_tmdb,...,runtime,status,tagline,vote_average,vote_count,credits,keywords,date_difference,region,anime
3119,In His Chart,23384864,23384864,Toho Company,2011-08-27,2011,inhischart,False,1442.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Unknown,0.0
305,Yomigaeri,23259542,23259542,-,2003-01-18,2003,yomigaeri,False,134.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Unknown,0.0
3120,Andalucia: Revenge of the Goddess,22929221,22929221,Toho Company,2011-06-25,2011,andalucia:revengeofthegoddess,False,1443.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Unknown,0.0
6662,Arashi Anniversary Tour 5 x 20 Film: Record of...,22677901,38052695,Shochiku,2021-11-26,2021,arashianniversarytour5x20film:recordofmemories,False,3192.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Unknown,0.0
4888,Botanical Books,20071961,20071961,Shochiku,2016-06-04,2016,botanicalbooks,False,2333.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Unknown,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6595,Kim Ji-young: Born 1982,128725,1448424,The Klockworx,2020-10-09,2020,kimji-young:born1982,False,3163.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Unknown,0.0
4862,Kanashimi no wasurekata: Documentary of Nogiza...,121285,309932,-,2015-07-10,2015,kanashiminowasurekata:documentaryofnogizaka46,False,2310.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Unknown,0.0
3067,Soup Opera,110362,110362,-,2010-10-02,2010,soupopera,False,1416.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Unknown,0.0
5608,Leap!,93715,1506409,-,2017-08-12,2017,leap!,False,2668.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Unknown,0.0


In [594]:
# show unique genre name separated by -
merged['genres'].str.split('-', expand=True).stack().value_counts()

Drama                              209
Drama,Romance                       80
Comedy                              57
Action,Adventure                    55
Comedy,Drama                        42
                                  ... 
Action,History,Drama                 1
Action,Mystery,ScienceFiction        1
Mystery,ScienceFiction,Thriller      1
Animation,Adventure                  1
Documentary,History                  1
Length: 906, dtype: int64

## Cleaning genres

In [572]:
merged['genres'] = merged['genres'].str.replace("Sci-Fi", "Science Fiction").str.replace("Anime|anime", "Animation").str.replace("-", ",").str.replace(" ", "")

C:\Users\MICROS~1\AppData\Local\Temp/ipykernel_8828/3343247779.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  merged['genres'] = merged['genres'].str.replace("Sci-Fi", "Science Fiction").str.replace("Anime|anime", "Animation").str.replace("-", ",").str.replace(" ", "")


## New columns "region" "anime"

In [581]:
# merged['jp']: 'jp', 'other', 'unknown'ArithmeticError
merged['region'] = np.nan
for idx, row in merged.iterrows():
    if row['original_language'] == 'ja':
        merged.loc[idx, 'region'] = 'Japan'
    elif pd.isna(row['original_language']):
        merged.loc[idx, 'region'] = 'Unknown'
    else:
        merged.loc[idx, 'region'] = 'Foreign'

#  Make a new column 'anime'. If merged['genres'] contains 'Animation', 'anime'=1, else 0
merged['anime'] = np.nan
for row in merged.itertuples():
    if pd.notna(row.genres) and 'Animation' in row.genres:
        merged.loc[row.Index, 'anime'] = 1
    else:
        merged.loc[row.Index, 'anime'] = 0

In [582]:
merged[merged['region']=='Unknown']

,original_title,gross,total_gross,distributor,JP_release_date,JP_release_year,revised_title,re_released,id_movies,id_tmdb,...,runtime,status,tagline,vote_average,vote_count,credits,keywords,date_difference,region,anime
45,Toei Spring Anime Fair,13107237,13107237,-,2002-03-02,2002,toeispringanimefair,False,23.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Unknown,0.0
46,Mohou-han,11702775,11702775,-,2002-06-08,2002,mohou-han,False,24.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Unknown,0.0
83,Totsunyûseyo! 'Asama Sansô' jiken,6394240,6394240,-,2002-05-11,2002,totsunyûseyo!'asamasansô'jiken,False,39.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Unknown,0.0
123,Madness in Bloom,3265669,3265669,-,2002-10-19,2002,madnessinbloom,False,60.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Unknown,0.0
124,E.T. the Extra-Terrestrial20th Anniversary,3259178,5837357,-,2002-04-27,2002,e.t.theextra-terrestrial20thanniversary,False,61.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Unknown,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7162,To the Solitary Me Who Loved You,288905,368027,Toei Company,2022-10-07,2022,tothesolitarymewholovedyou,False,3441.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Unknown,0.0
7191,Fruits Basket: Prelude,274015,434280,Avex Entertainment,2022-02-18,2022,fruitsbasket:prelude,False,3445.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Unknown,0.0
7193,Thunderbirds Go Go,242758,331934,-,2022-01-07,2022,thunderbirdsgogo,False,3447.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Unknown,0.0
7194,Miyuki Nakajima Live History 2007-2016,227387,343339,-,2022-01-21,2022,miyukinakajimalivehistory2007-2016,False,3448.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Unknown,0.0


In [583]:
merged.to_csv('merged.csv', index=False)

In [584]:
# merge "rankng" and "merged" dataframes.  
merged_ranking = pd.merge(ranking, merged, left_on = 'id', right_on = 'id_movies')
# merged_rankingの_yで終わる列はすべて削除
merged_ranking.drop(merged_ranking.columns[merged_ranking.columns.str.endswith('_y')], axis=1, inplace=True)

# _xという列の_xを削除
merged_ranking.rename(columns=lambda x: x.replace('_x', ''), inplace=True)

In [585]:
merged_ranking

,year,rank,original_title,gross,total_gross,release_date,distributor,order,revised_release_date,release_year,...,runtime,status,tagline,vote_average,vote_count,credits,keywords,date_difference,region,anime
0,2002,1,Star Wars: Episode II - Attack of the Clones,78182528,78182528,Jul 13,-,1,2002-07-13,2002,...,142.0,Released,A Jedi Shall Not Know Anger. Nor Hatred. Nor L...,6.548,11931.0,Hayden Christensen-Ewan McGregor-Natalie Portm...,laser gun-senate-investigation-army-cult figur...,59 days,Foreign,0.0
1,2002,2,"Monsters, Inc.",74437612,74437612,Mar 2,-,1,2002-03-02,2002,...,92.0,Released,We Scare Because We Care.,7.831,16786.0,John Goodman-Billy Crystal-Mary Gibbs-Steve Bu...,monster-cheating-kidnapping-infant-villain-pan...,121 days,Foreign,1.0
2,2002,3,The Lord of the Rings: The Fellowship of the Ring,69140370,69140370,Mar 2,-,1,2002-03-02,2002,...,179.0,Released,One ring to rule them all,8.399,22910.0,Elijah Wood-Ian McKellen-Viggo Mortensen-Sean ...,elves-dwarf-orcs-loss of loved one-based on no...,74 days,Foreign,0.0
3,2002,4,Harry Potter and the Chamber of Secrets,67578962,142786414,Nov 23,-,1,2002-11-23,2002,...,161.0,Released,Hogwarts is back in session.,7.722,19555.0,Daniel Radcliffe-Rupert Grint-Emma Watson-Kenn...,flying car-witch-sword-magic-diary-child hero-...,10 days,Foreign,0.0
4,2003,7,Harry Potter and the Chamber of Secrets,34854123,142786414,Nov 23,-,2,2002-11-23,2002,...,161.0,Released,Hogwarts is back in session.,7.722,19555.0,Daniel Radcliffe-Rupert Grint-Emma Watson-Kenn...,flying car-witch-sword-magic-diary-child hero-...,10 days,Foreign,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3458,2022,154,The Forever Purge,65542,80624,May 20,-,1,2022-05-20,2022,...,103.0,Released,The rules are broken.,7.116,2004.0,Ana de la Reguera-Tenoch Huerta Mejía-Josh Luc...,sequel-prequel-murder-purge,324 days,Foreign,0.0
3459,2022,155,Firestarter,58262,73364,Jun 17,Toho-Towa,1,2022-06-17,2022,...,94.0,Released,Never play with fire.,5.808,465.0,Ryan Kiera Armstrong-Zac Efron-Sydney Lemmon-K...,fire-based on novel or book-telepathy-psychic ...,36 days,Foreign,0.0
3460,2022,156,The Nest,49732,49732,Apr 29,Kino Films,1,2022-04-29,2022,...,7.0,Released,Some things are best left in the woods.,0.000,0.0,Lauren Tyrrell-Samantha Brown,alien life-form-forest-mystery-egg,145 days,Foreign,0.0
3461,2022,157,Be Natural: The Untold Story of Alice Guy-Blaché,15094,15094,Jul 23,-,1,2022-07-23,2022,...,103.0,Released,She saw the future through the camera lens,7.100,25.0,Jodie Foster-Richard Abel-Marc Abraham-Stephan...,based on novel or book-movie business-woman di...,1324 days,Foreign,0.0


In [561]:
merged_ranking.to_csv('merged_ranking.csv', index=False)

# Year dataset
This dataset is from http://www.eiren.org/toukei/data.html


In [3]:
year = pd.read_csv('year_analysis.csv')
# Rename column "Screens" with "Theaters"
year.rename(columns={'Screens':'Total_Screens'}, inplace=True)
year.rename(columns={'thousand people':'Moviegoers'}, inplace=True)
year.rename(columns={'yen': 'ticket_price'}, inplace=True)

# In the Theater column, there are two values in each row separated by ().
# I want to keep the first value in the column and the second value in a new column "Screens"
year['Total_Screens'] = year['Total_Screens'].str.replace(",", "").str.replace("\（", "(").str.replace("\）", ")")
year['Cinecons_Screens'] = year['Total_Screens'].str.extract(r'\((.*?)\)')
year['Total_Screens'] = year['Total_Screens'].str.replace(r'\(.*?\)', '')
year['Non_Cinecons_Screens'] = year['Total_Screens'] = year['Cinecons_Screens']

year.drop(columns=['Unnamed: 12', 'Unnamed: 13'], inplace=True)
year
# change the column order
# year.columns = ['Year', 'Total_Screens', 'Cinecons_Screens', 'Non_Cinecons_Screens', 'total_movies', 'JP_movies', 'foreign_movies', 'moviegoers', 'ticket_price', 'Total_revenue', "JP_revenue", 'Foreign_revenue', 'JP_share', 'Foreign_share'] 

C:\Users\MICROS~1\AppData\Local\Temp/ipykernel_15296/3644410473.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  year['Total_Screens'] = year['Total_Screens'].str.replace(",", "").str.replace("\（", "(").str.replace("\）", ")")
C:\Users\MICROS~1\AppData\Local\Temp/ipykernel_15296/3644410473.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  year['Total_Screens'] = year['Total_Screens'].str.replace(r'\(.*?\)', '')


,Year,Total_Screens,JP_movies,foreign_movies,total_movies,Moviegoers,ticket_price,JP_revenue,Foreign_revenue,Total_revenue,JP_share,Foreign_share,Cinecons_Screens,Non_Cinecons_Screens
0,2000,1123,282,362,644,"135,390","1,262","54,334","116,528","170,862",31.8,68.2,1123,1123
1,2001,1259,281,349,630,"163,280","1,226","78,144","122,010","200,154",39.0,61.0,1259,1259
2,2002,1396,293,347,640,"160,767","1,224","53,294","143,486","196,780",27.1,72.9,1396,1396
3,2003,1533,287,335,622,"162,347","1,252","67,125","136,134","203,259",33.0,67.0,1533,1533
4,2004,1766,310,339,649,"170,092","1,240","79,054","131,860","210,914",37.5,62.5,1766,1766
5,2005,1954,356,375,731,"160,453","1,235","81,780","116,380","198,160",41.3,58.7,1954,1954
6,2006,2230,417,404,821,"164,585","1,233","107,944","94,990","202,934",53.2,46.8,2230,2230
7,2007,2454,407,403,810,"163,193","1,216","94,645","103,798","198,443",47.7,52.3,2454,2454
8,2008,2659,418,388,806,"160,491","1,214","115,859","78,977","194,836",59.5,40.5,2659,2659
9,2009,2723,448,314,762,"169,297","1,217","117,309","88,726","206,035",56.9,43.1,2723,2723


In [4]:
# all object columns are converted to numeric columns

year[year.select_dtypes(['object']).columns] = year[year.select_dtypes(['object']).columns].apply(lambda x: x.str.replace('[^0-9]', '', regex=True).astype(int), axis=1)
year

,Year,Total_Screens,JP_movies,foreign_movies,total_movies,Moviegoers,ticket_price,JP_revenue,Foreign_revenue,Total_revenue,JP_share,Foreign_share,Cinecons_Screens,Non_Cinecons_Screens
0,2000,1123,282,362,644,135390,1262,54334,116528,170862,31.8,68.2,1123,1123
1,2001,1259,281,349,630,163280,1226,78144,122010,200154,39.0,61.0,1259,1259
2,2002,1396,293,347,640,160767,1224,53294,143486,196780,27.1,72.9,1396,1396
3,2003,1533,287,335,622,162347,1252,67125,136134,203259,33.0,67.0,1533,1533
4,2004,1766,310,339,649,170092,1240,79054,131860,210914,37.5,62.5,1766,1766
5,2005,1954,356,375,731,160453,1235,81780,116380,198160,41.3,58.7,1954,1954
6,2006,2230,417,404,821,164585,1233,107944,94990,202934,53.2,46.8,2230,2230
7,2007,2454,407,403,810,163193,1216,94645,103798,198443,47.7,52.3,2454,2454
8,2008,2659,418,388,806,160491,1214,115859,78977,194836,59.5,40.5,2659,2659
9,2009,2723,448,314,762,169297,1217,117309,88726,206035,56.9,43.1,2723,2723


In [5]:
year

,Year,Total_Screens,JP_movies,foreign_movies,total_movies,Moviegoers,ticket_price,JP_revenue,Foreign_revenue,Total_revenue,JP_share,Foreign_share,Cinecons_Screens,Non_Cinecons_Screens
0,2000,1123,282,362,644,135390,1262,54334,116528,170862,31.8,68.2,1123,1123
1,2001,1259,281,349,630,163280,1226,78144,122010,200154,39.0,61.0,1259,1259
2,2002,1396,293,347,640,160767,1224,53294,143486,196780,27.1,72.9,1396,1396
3,2003,1533,287,335,622,162347,1252,67125,136134,203259,33.0,67.0,1533,1533
4,2004,1766,310,339,649,170092,1240,79054,131860,210914,37.5,62.5,1766,1766
5,2005,1954,356,375,731,160453,1235,81780,116380,198160,41.3,58.7,1954,1954
6,2006,2230,417,404,821,164585,1233,107944,94990,202934,53.2,46.8,2230,2230
7,2007,2454,407,403,810,163193,1216,94645,103798,198443,47.7,52.3,2454,2454
8,2008,2659,418,388,806,160491,1214,115859,78977,194836,59.5,40.5,2659,2659
9,2009,2723,448,314,762,169297,1217,117309,88726,206035,56.9,43.1,2723,2723


In [6]:
year.to_csv("revised_year.csv")